In [ ]:
select * from Szallashely for xml auto

In [ ]:
select sz.szoba_id, sz.szoba_szama, szh.szallas_nev
from Szallashely szh inner join Szoba sz on szh.szallas_id = sz.szallas_fk
for xml auto, elements

In [ ]:
select f.foglalas_pk as [@azon], v.nev as [@nev], f.mettol, f.felnott_szam
from Foglalas f inner join Vendeg v on f.ugyfel_fk = v.USERNEV
order by f.foglalas_pk ASC
for xml path('Foglalas'), root('Foglalasok')

In [ ]:
declare @xml XML
set @xml = cast('<Foglalasok>
	<Foglalas azon="558" nev="Fő Nándor">
		<mettol>
			2016-04-06
		</mettol>
		<felnott_szam>
			2
		</felnott_szam>
	</Foglalas>
    <Foglalas azon="559" nev="Kelemen Áron">
		<mettol>
			2016-04-06
		</mettol>
		<felnott_szam>
			2
		</felnott_szam>
	</Foglalas>
    </Foglalasok>' as xml)

select @xml.query('Foglalasok/Foglalas')

In [ ]:
declare @xml XML
set @xml = cast('<Foglalasok>
	<Foglalas azon="558">
    <Vendeg nev="Fő Nándor" />
		<mettol>
			2016-04-06
		</mettol>
		<felnott_szam>
			2
		</felnott_szam>
	</Foglalas>
    <Foglalas azon="559">
    <Vendeg nev="Kelemen Áron" />
		<mettol>
			2016-04-06
		</mettol>
		<felnott_szam>
			2
		</felnott_szam>
	</Foglalas>
    </Foglalasok>' as xml)

    select @xml.value('(/Foglalasok/Foglalas/Vendeg/@nev)[1]','nvarchar')

    select f.value('@foglalas_pk', 'int') as 'azonositok' 
    from @xml.nodes('/Foglalasok/Foglalas') as t(f)

In [ ]:
declare @idoc int
declare @xml XML
set @xml = cast('<Foglalasok>
	<Foglalas azon="558">
    <Vendeg nev="Fő Nándor" />
		<mettol>
			2016-04-06
		</mettol>
		<felnott_szam>
			2
		</felnott_szam>
	</Foglalas>
    <Foglalas azon="559">
    <Vendeg nev="Kelemen Áron" />
		<mettol>
			2016-04-06
		</mettol>
		<felnott_szam>
			2
		</felnott_szam>
	</Foglalas>
    </Foglalasok>' as xml)

    -- xml betoltes
    exec sp_xml_preparedocument @idoc OUTPUT, @xml

    select * from OPENXML(@idoc, 'Fogalalasok/Foglalas', 2)
    with(foglalas_pk int '@foglalas_pk', mettol date '@mettol', felnott_szam int '@felnott_szam')

    -- memoriafelszabaditas
    exec sp_xml_removedocument @idoc

In [30]:
declare @i GEOGRAPHY
declare @j GEOGRAPHY

set @i = geography::STGeomFromText('POINT(-122.34900 47.65100)', 4326)
set @j = geography::STGeomFromText('POINT(-100.10 55.35)', 4326)

select @i.STDistance(@j)

(1 row affected)

Total execution time: 00:00:00.049

(No column name)
1755449.3151604538
